In [1]:
# Modification of technicals function
# Now take dataframes as input, instead of portfolios

# Author Yansheng Zhu
# Time last edited: 15:40 July 13, 2020
# Last edited by: Yansheng Zhu

# WARNING: The default intervals are recommended on daily frequency. 
#          When handling with monthly data,it's better to arrange a shorter interval, 
#          in case of there's no enough data to calculate the technicals


# Technicals included: Moving Average
#                      Moving Average Convergence Divergence
#                      Exponential Moving Average
#                      Relative Strength Index
#                      Standard Deviation 
#                      Williams %R
#                      Average True Range

import copy
import numpy as np
import pandas as pd


In [2]:
#demo dataframe
df=pd.read_csv('data/1d/AKAM.csv')

In [3]:
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2000-01-03,344.19,345.50,311.00,321.25,285600,0,0
1,2000-01-04,320.00,321.50,300.00,300.00,215900,0,0
2,2000-01-05,282.50,299.62,262.00,283.50,1183500,0,0
3,2000-01-06,272.00,278.12,235.25,236.12,797500,0,0
4,2000-01-07,248.38,254.50,238.00,248.38,729100,0,0
...,...,...,...,...,...,...,...,...
5162,2020-07-10,115.30,115.30,113.37,114.26,1262500,0,0
5163,2020-07-13,115.12,115.24,109.81,109.94,1599900,0,0
5164,2020-07-14,109.97,112.36,108.70,112.27,2089800,0,0
5165,2020-07-15,111.78,113.65,111.11,112.78,2232700,0,0


In [4]:

##function to construct technical: MA
#input: dataframe(df) that stores the data with Close Price , interval (5 10 30 60 for MA5 MA10...), 
#       new = True for returning a new df deepcopy, new = False for no deepcopy
#output: df, with updated feature MA

def build_MA(df,interval=30, new = False):
    if new == True:
        df = copy.deepcopy(df) 
    col_name='MA'+str(interval)  
    #calculate MA from an available date. 
    #For example, MA30 will be calculated from 30th row in the dataframe
    #to make sure there're enough days for MA
    starting=interval
        
    #initialization
    df[col_name]=0
    
    for i in range(starting,len(df)):
        
        df.loc[i,col_name]=df['Close'][(i - interval): i].mean()
    return df


In [5]:
build_MA(df)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA30
0,2000-01-03,344.19,345.50,311.00,321.25,285600,0,0,0.000000
1,2000-01-04,320.00,321.50,300.00,300.00,215900,0,0,0.000000
2,2000-01-05,282.50,299.62,262.00,283.50,1183500,0,0,0.000000
3,2000-01-06,272.00,278.12,235.25,236.12,797500,0,0,0.000000
4,2000-01-07,248.38,254.50,238.00,248.38,729100,0,0,0.000000
...,...,...,...,...,...,...,...,...,...
5162,2020-07-10,115.30,115.30,113.37,114.26,1262500,0,0,104.381333
5163,2020-07-13,115.12,115.24,109.81,109.94,1599900,0,0,104.759667
5164,2020-07-14,109.97,112.36,108.70,112.27,2089800,0,0,104.897667
5165,2020-07-15,111.78,113.65,111.11,112.78,2232700,0,0,105.116333


In [6]:
df['MA30'][:60]

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
5       0.000000
6       0.000000
7       0.000000
8       0.000000
9       0.000000
10      0.000000
11      0.000000
12      0.000000
13      0.000000
14      0.000000
15      0.000000
16      0.000000
17      0.000000
18      0.000000
19      0.000000
20      0.000000
21      0.000000
22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
27      0.000000
28      0.000000
29      0.000000
30    252.612000
31    249.320333
32    247.524333
33    246.474333
34    246.660000
35    246.451333
36    245.336667
37    245.215667
38    245.651000
39    246.449000
40    247.490667
41    249.728000
42    251.169667
43    252.315667
44    251.815667
45    251.828333
46    251.351333
47    251.634667
48    252.130667
49    252.620333
50    253.308000
51    253.347667
52    253.197667
53    252.316333
54    251.591333
55    250.554000
56    249.472667
57    248.372667
58    247.7726

In [7]:
#function to construct technical: MACD
#input: dataframe(df) that stores the data with Close Price, 
#       new = True for returning a new df deepcopy, new = False for no deepcopy
#       accuracy means how much weight to be included, default 0.001 means 99.90% weight to be included 
#output: df, with updated feature MACD


def build_MACD(df, new = False, accuracy=0.001):
    
    if new == True:
        df = copy.deepcopy(df)
    
    col_name='MACD'
    
    #calculate EMA from an available month. 
    #if daily data from 2020-01-01, accuracy = 0.001, then 89.76=np.log(0.001)/np.log(25/27)
    #lagged terms will be required in calculation of EMA26, which will be calculated from 2020-07-01 
    #to make sure there're enough days for EMA26 (the longest period used in MACD calcculation)
    required_lag= np.ceil(np.log(accuracy)/np.log(25/27))
    
    starting=int(required_lag)
    #initialization
    df[col_name]=0
    
    if starting >= len(df):
        print('no enough data for constructing MACD')
        return df
    #need to make sure 
    
    for i in range(starting,len(df)):
            
        lag_26=int(np.log(accuracy)/np.log(25/27)+1)
        lag_12=int(np.log(accuracy)/np.log(11/13)+1)
        lag_9=int(np.log(accuracy)/np.log(8/10)+1)
            
        EMA26=EMA_calculation(26, df['Close'][(i - lag_26):i], lag_26)
        EMA12=EMA_calculation(12, df['Close'][(i - lag_12):i], lag_12)
        EMA9=EMA_calculation(9, df['Close'][(i - lag_9):i], lag_9)
        
        df.loc[i,col_name]=224/51*EMA9-16/3*EMA12+16/17*EMA26
    return df



#function to calculate EMA, used in MACD and EMA technicals construction
#input: interval, the EMA period length to be calculated (not the exact length, interval=30 doesnt mean use only 30 price to calculate)
#       prices, the prices used in calculation. Can be list or array
#       lag, the exact length needed in calculation. Determined by 'interval' and the accuracy       
#output: EMA
def EMA_calculation(interval,prices,lag):
    coefs=2/(interval+1)*np.array([ ((interval-1)/(interval+1))**i for i in range(0,lag)])
    coefs=coefs[::-1]
    EMA=np.dot(coefs,prices)
    return EMA


In [8]:
build_MACD(df)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA30,MACD
0,2000-01-03,344.19,345.50,311.00,321.25,285600,0,0,0.000000,0.000000
1,2000-01-04,320.00,321.50,300.00,300.00,215900,0,0,0.000000,0.000000
2,2000-01-05,282.50,299.62,262.00,283.50,1183500,0,0,0.000000,0.000000
3,2000-01-06,272.00,278.12,235.25,236.12,797500,0,0,0.000000,0.000000
4,2000-01-07,248.38,254.50,238.00,248.38,729100,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
5162,2020-07-10,115.30,115.30,113.37,114.26,1262500,0,0,104.381333,2.129809
5163,2020-07-13,115.12,115.24,109.81,109.94,1599900,0,0,104.759667,2.017743
5164,2020-07-14,109.97,112.36,108.70,112.27,2089800,0,0,104.897667,1.239375
5165,2020-07-15,111.78,113.65,111.11,112.78,2232700,0,0,105.116333,0.932850


In [9]:
#MACD data starts from 90th row, because of the accuracy requirement 
df['MACD'][50:100]

50    0.000000
51    0.000000
52    0.000000
53    0.000000
54    0.000000
55    0.000000
56    0.000000
57    0.000000
58    0.000000
59    0.000000
60    0.000000
61    0.000000
62    0.000000
63    0.000000
64    0.000000
65    0.000000
66    0.000000
67    0.000000
68    0.000000
69    0.000000
70    0.000000
71    0.000000
72    0.000000
73    0.000000
74    0.000000
75    0.000000
76    0.000000
77    0.000000
78    0.000000
79    0.000000
80    0.000000
81    0.000000
82    0.000000
83    0.000000
84    0.000000
85    0.000000
86    0.000000
87    0.000000
88    0.000000
89    0.000000
90    8.534859
91    6.772333
92    5.373697
93    5.126791
94    5.231157
95    5.523907
96    5.577673
97    5.184516
98    4.648250
99    4.156275
Name: MACD, dtype: float64

In [10]:

#function to construct technical: EMA
#input: dataframe(df) that stores the data with Close Price, 
#       interval, a int to specify the EMA period length (not the exact length, interval=30 doesnt mean use only 30 price to calculate)
#        new = True for returning a new df deepcopy, new = False for no deepcopy
#       accuracy means how much weight to be included, default 0.001 means 99.90% weight to be included 
#output: df, with updated feature EMA 

def build_EMA(df, interval=30, new = False, accuracy=0.001): 
    if new == True:
        df = copy.deepcopy(df)
 
    col_name='EMA'+str(interval)
            
    #calculate EMA from an available month. 
    #if daily data from 2010-01-01, accuracy = 0.001, interval=30, then 103.58=np.log(0.001)/np.log(29/31)
    #lagged terms will be required in calculation of EMA30, which will be calculated from 2010-08-01 
    #to make sure there're enough days for EMA30 
    required_lag= np.ceil(np.log(accuracy)/np.log((interval-1)/(interval+1)))
            
    starting=int(required_lag)
    #initialization        
    df[col_name]=0
    
    if starting >= len(df):
        print('no enough data for constructing EMA'+str(interval))
        return df
            
    for i in range(starting,len(df)):
        EMA=EMA_calculation(interval, df['Close'][(i - starting):i], int(required_lag))
        df.loc[i,col_name]=EMA
    return df

In [11]:
build_EMA(df)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA30,MACD,EMA30
0,2000-01-03,344.19,345.50,311.00,321.25,285600,0,0,0.000000,0.000000,0.000000
1,2000-01-04,320.00,321.50,300.00,300.00,215900,0,0,0.000000,0.000000,0.000000
2,2000-01-05,282.50,299.62,262.00,283.50,1183500,0,0,0.000000,0.000000,0.000000
3,2000-01-06,272.00,278.12,235.25,236.12,797500,0,0,0.000000,0.000000,0.000000
4,2000-01-07,248.38,254.50,238.00,248.38,729100,0,0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5162,2020-07-10,115.30,115.30,113.37,114.26,1262500,0,0,104.381333,2.129809,105.075722
5163,2020-07-13,115.12,115.24,109.81,109.94,1599900,0,0,104.759667,2.017743,105.662211
5164,2020-07-14,109.97,112.36,108.70,112.27,2089800,0,0,104.897667,1.239375,105.932085
5165,2020-07-15,111.78,113.65,111.11,112.78,2232700,0,0,105.116333,0.932850,106.334735


In [12]:
#EMA30 starts from 104th row
df['EMA30'][100:200]


100     0.000000
101     0.000000
102     0.000000
103     0.000000
104    86.707059
         ...    
195    56.779424
196    55.805361
197    55.139681
198    54.343963
199    53.994362
Name: EMA30, Length: 100, dtype: float64

In [13]:

#function to construct technical: RSI
#input: dataframe(df) that stores the data with Close Price, 
#       interval, a int to specify the RSI period length (not the exact length, interval=30 doesnt mean use only 30 price to calculate), default = 14(most common period)
#       new = True for returning a new df deepcopy, new = False for no deepcopy
#       accuracy means how much weight to be included, default 0.001 means 99.90% weight to be included 
#output: df, with updated feature RSI


def build_RSI(df, interval=14, new = False, accuracy=0.001): 
    if new == True:
        df = copy.deepcopy(df)
    
    col_name='RSI'+str(interval)
        
    required_lag= np.ceil(np.log(accuracy)/np.log(1-1/interval))
    
    starting=int(required_lag)
    
    #initialization        
    df[col_name]=0
    if starting >= len(df):
        print('no enough data for constructing RSI'+str(interval))
        return df
    
    # two factors for exponential weights
    alpha_1=1/interval
    alpha_2=1-alpha_1
            
    for i in range(starting+1, len(df)):
        #
        price_list=df['Close'][(i-starting-1):i].reset_index(drop=True)
        price_changes=(price_list-price_list.shift())[1:]
                
        U=[round(ii,ndigits=2) if ii > 0  else 0 for ii in price_changes][::-1]
        D=[round(-ii,ndigits=2) if ii < 0 else 0 for ii in price_changes][::-1]
              
        #RSI uses a different exponential weight method (SMMA) compared with MACD and EMA
        coefs=[alpha_1*(alpha_2** ii ) for ii in range(0,int(required_lag))]
                
        SMMA_U=np.dot(U,coefs)
        SMMA_D=np.dot(D,coefs)
                
        RSI=100*SMMA_U/(SMMA_U+SMMA_D)
             
        df.loc[i,col_name]=RSI
    return df



In [14]:
build_RSI(df)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA30,MACD,EMA30,RSI14
0,2000-01-03,344.19,345.50,311.00,321.25,285600,0,0,0.000000,0.000000,0.000000,0.000000
1,2000-01-04,320.00,321.50,300.00,300.00,215900,0,0,0.000000,0.000000,0.000000,0.000000
2,2000-01-05,282.50,299.62,262.00,283.50,1183500,0,0,0.000000,0.000000,0.000000,0.000000
3,2000-01-06,272.00,278.12,235.25,236.12,797500,0,0,0.000000,0.000000,0.000000,0.000000
4,2000-01-07,248.38,254.50,238.00,248.38,729100,0,0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
5162,2020-07-10,115.30,115.30,113.37,114.26,1262500,0,0,104.381333,2.129809,105.075722,70.867387
5163,2020-07-13,115.12,115.24,109.81,109.94,1599900,0,0,104.759667,2.017743,105.662211,70.927389
5164,2020-07-14,109.97,112.36,108.70,112.27,2089800,0,0,104.897667,1.239375,105.932085,57.943288
5165,2020-07-15,111.78,113.65,111.11,112.78,2232700,0,0,105.116333,0.932850,106.334735,61.994711


In [15]:
df['RSI14'][90:120]


90      0.000000
91      0.000000
92      0.000000
93      0.000000
94      0.000000
95     37.702655
96     37.097285
97     35.537436
98     34.448873
99     33.718026
100    33.086187
101    31.259214
102    30.147232
103    43.354654
104    39.007473
105    44.224436
106    53.320891
107    56.388950
108    51.823916
109    50.619707
110    53.479084
111    57.399008
112    51.506012
113    56.338999
114    53.852377
115    52.206052
116    50.205161
117    53.036112
118    53.789207
119    64.258844
Name: RSI14, dtype: float64

In [16]:

#function to construct technical: std
#input: dataframe(df) that stores the data with Close Price, 
#       interval, a int to specify the std period length, default = 10
#       new = True for returning a new df deepcopy, new = False for no deepcopy
#output: df, with updated feature std


def build_std(df, interval=10, new = False): 
    if new == True:
        df = copy.deepcopy(df)
    
    col_name='std'+str(interval)
    
    starting=interval
    
    #initialization        
    df[col_name]=0
    if starting >= len(df):
        print('no enough data for constructing W%R'+str(interval))
        return df

    for i in range(starting, len(df)):
        #
        std=df['Close'][(i-interval):i].std()
        df.loc[i,col_name]=std
    
    return df


In [17]:
build_std(df)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA30,MACD,EMA30,RSI14,std10
0,2000-01-03,344.19,345.50,311.00,321.25,285600,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
1,2000-01-04,320.00,321.50,300.00,300.00,215900,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
2,2000-01-05,282.50,299.62,262.00,283.50,1183500,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
3,2000-01-06,272.00,278.12,235.25,236.12,797500,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
4,2000-01-07,248.38,254.50,238.00,248.38,729100,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5162,2020-07-10,115.30,115.30,113.37,114.26,1262500,0,0,104.381333,2.129809,105.075722,70.867387,4.109650
5163,2020-07-13,115.12,115.24,109.81,109.94,1599900,0,0,104.759667,2.017743,105.662211,70.927389,3.782957
5164,2020-07-14,109.97,112.36,108.70,112.27,2089800,0,0,104.897667,1.239375,105.932085,57.943288,3.395685
5165,2020-07-15,111.78,113.65,111.11,112.78,2232700,0,0,105.116333,0.932850,106.334735,61.994711,2.778635


In [18]:
#function to construct technical: W%R
#input: dataframe(df) that stores the data with Close Price, 
#       interval, a int to specify the W%R period length, default = 30(most common period)
#       new = True for returning a new df deepcopy, new = False for no deepcopy
#output: df, with updated feature W%R


def build_WpercentR(df, interval=30, new = False): 
    if new == True:
        df = copy.deepcopy(df)
    
    col_name='W%R'+str(interval)
    
    starting=interval
    
    #initialization        
    df[col_name]=0
    if starting >= len(df):
        print('no enough data for constructing W%R'+str(interval))
        return df

    for i in range(starting, len(df)):
        #
        
        maxi=df['Close'][(i-interval):i].max()
        mini=df['Close'][(i-interval):i].min()
                
        WR=-100*(maxi-df['Close'][i-1])/(maxi-mini)
        df.loc[i,col_name]=WR
    
    return df

In [19]:
build_WpercentR(df)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA30,MACD,EMA30,RSI14,std10,W%R30
0,2000-01-03,344.19,345.50,311.00,321.25,285600,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2000-01-04,320.00,321.50,300.00,300.00,215900,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2000-01-05,282.50,299.62,262.00,283.50,1183500,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2000-01-06,272.00,278.12,235.25,236.12,797500,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2000-01-07,248.38,254.50,238.00,248.38,729100,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5162,2020-07-10,115.30,115.30,113.37,114.26,1262500,0,0,104.381333,2.129809,105.075722,70.867387,4.109650,-0.000000
5163,2020-07-13,115.12,115.24,109.81,109.94,1599900,0,0,104.759667,2.017743,105.662211,70.927389,3.782957,-0.000000
5164,2020-07-14,109.97,112.36,108.70,112.27,2089800,0,0,104.897667,1.239375,105.932085,57.943288,3.395685,-28.309305
5165,2020-07-15,111.78,113.65,111.11,112.78,2232700,0,0,105.116333,0.932850,106.334735,61.994711,2.778635,-13.040629


In [20]:
df['W%R30'][:50]

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
5      0.000000
6      0.000000
7      0.000000
8      0.000000
9      0.000000
10     0.000000
11     0.000000
12     0.000000
13     0.000000
14     0.000000
15     0.000000
16     0.000000
17     0.000000
18     0.000000
19     0.000000
20     0.000000
21     0.000000
22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
27     0.000000
28     0.000000
29     0.000000
30   -82.797267
31   -87.570621
32   -53.216216
33   -45.270270
34   -59.202703
35   -58.621622
36   -41.740879
37    -9.063488
38   -20.195346
39   -35.004309
40   -28.540649
41    -4.264756
42   -22.381440
43   -21.733197
44   -46.264074
45   -31.866257
46   -27.499147
47    -0.000000
48    -0.000000
49   -15.306122
Name: W%R30, dtype: float64

In [21]:

#function to construct technical: ATR
#input: dataframe(df) that stores the data with Close Price, 
#       interval, a int to specify the ATR period length (not the exact length, interval=30 doesnt mean use only 30 price to calculate), default = 30(most common period)
#       new = True for returning a new df deepcopy, new = False for no deepcopy
#       accuracy means how much weight to be included, default 0.001 means 99.90% weight to be included 
#output: df, with updated feature ATR

def build_ATR(df, interval=30, new = False, accuracy=0.001): 
    if new == True:
        df= copy.deepcopy(df)

    
    col_name='ATR'+str(interval)
    
    required_lag= np.ceil(np.log(accuracy)/np.log(1-1/int(interval)))
    
    starting=int(required_lag)
    
    #initialization     
    df[col_name]=0
    
    if starting >= len(df):
        print('no enough data for constructing ATR'+str(interval))
        return df
    
    # two factors for exponential weights       
    alpha_1=1/int(interval)
    alpha_2=1-alpha_1
            
    for i in range(starting+1,len(df)):
        #
        close_list=df['Close'][(i-starting-1):i].reset_index(drop=True)
        high_list=df['High'][(i-starting-1):i].reset_index(drop=True)
        low_list=df['Low'][(i-starting-1):i].reset_index(drop=True)
                
        TR_list=[max(high_list[ii],close_list[ii-1])-min(low_list[ii],close_list[ii-1]) for ii in range(1,len(close_list))]
                
        coefs=[alpha_1*(alpha_2** ii ) for ii in range(0,int(required_lag))]
                
        ATR=np.dot(coefs,TR_list)
             
        df.loc[i,col_name]=ATR  
    return df

In [22]:
build_ATR(df)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA30,MACD,EMA30,RSI14,std10,W%R30,ATR30
0,2000-01-03,344.19,345.50,311.00,321.25,285600,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2000-01-04,320.00,321.50,300.00,300.00,215900,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2000-01-05,282.50,299.62,262.00,283.50,1183500,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2000-01-06,272.00,278.12,235.25,236.12,797500,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2000-01-07,248.38,254.50,238.00,248.38,729100,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5162,2020-07-10,115.30,115.30,113.37,114.26,1262500,0,0,104.381333,2.129809,105.075722,70.867387,4.109650,-0.000000,1.799989
5163,2020-07-13,115.12,115.24,109.81,109.94,1599900,0,0,104.759667,2.017743,105.662211,70.927389,3.782957,-0.000000,1.786606
5164,2020-07-14,109.97,112.36,108.70,112.27,2089800,0,0,104.897667,1.239375,105.932085,57.943288,3.395685,-28.309305,1.783571
5165,2020-07-15,111.78,113.65,111.11,112.78,2232700,0,0,105.116333,0.932850,106.334735,61.994711,2.778635,-13.040629,1.804164


In [23]:
df['ATR30'][200:250]

200     0.000000
201     0.000000
202     0.000000
203     0.000000
204     0.000000
205    23.813450
206    23.893465
207    23.407247
208    22.550840
209    22.694820
210    22.171773
211    21.876203
212    21.277204
213    20.558592
214    20.384189
215    20.311388
216    20.270512
217    20.426605
218    19.570754
219    19.355425
220    19.078959
221    19.256340
222    19.023911
223    18.643561
224    18.079732
225    17.976120
226    18.121964
227    18.113252
228    17.924529
229    17.887574
230    17.905290
231    18.143579
232    18.428673
233    18.551185
234    18.779227
235    18.875449
236    18.509274
237    18.544226
238    18.744219
239    19.196987
240    18.928038
241    18.776881
242    18.941758
243    18.948485
244    19.162427
245    18.907190
246    18.673484
247    18.602432
248    18.345409
249    18.488782
Name: ATR30, dtype: float64

In [29]:
# Function: Build up technical for stochastic oscillators
# Author: Zhishu Zhang
# Time last edited: July 5, 2020
# input: portfolio dictionary,
#       path is the location;
#       new is to make a deepcopy;
#       n is the months traced back, since we use monthly instead of daily data, we set n = 3.
# output: A portfolio
def s_oscillator(df, new = False, n=3):
    if new == True:
        df = copy.deepcopy(df)
    
    col_name_1 = 'stochastic_oscillator_k'
    col_name_2 = 'stochastic_oscillator_d'
    starting = n
    df[col_name_1] = 50
    df[col_name_2] = 50
    
    kn = 50
    dn = 50
    
    for i in range(starting, len(df)):
        Hn = df['Close'][(i-n):i].max()
        Ln = df['Close'][(i-n):i].min()
        
        Cn = df['Close'][i]
        RSV = (Cn-Ln)/(Hn-Ln)
            
        # Calculate Kn and Dn
        Kn = RSV/3 + 2/3*kn
        Dn = 2/3*Kn + dn/3
            
        
        df.loc[i,col_name_1] = Kn
        df.loc[i,col_name_2] = Dn
        
        kn = Kn
        dn = Dn
        
    return df


In [30]:
s_oscillator(df)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA30,MACD,EMA30,RSI14,std10,W%R30,ATR30,stochastic_oscillator_k,stochastic_oscillator_d
0,2000-01-03,344.19,345.50,311.00,321.25,285600,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,50.000000
1,2000-01-04,320.00,321.50,300.00,300.00,215900,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,50.000000
2,2000-01-05,282.50,299.62,262.00,283.50,1183500,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,50.000000
3,2000-01-06,272.00,278.12,235.25,236.12,797500,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.914967,38.609978
4,2000-01-07,248.38,254.50,238.00,248.38,729100,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.007285,27.541516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5162,2020-07-10,115.30,115.30,113.37,114.26,1262500,0,0,104.381333,2.129809,105.075722,70.867387,4.109650,-0.000000,1.799989,1.284462,1.313990
5163,2020-07-13,115.12,115.24,109.81,109.94,1599900,0,0,104.759667,2.017743,105.662211,70.927389,3.782957,-0.000000,1.786606,0.283981,0.627317
5164,2020-07-14,109.97,112.36,108.70,112.27,2089800,0,0,104.897667,1.239375,105.932085,57.943288,3.395685,-28.309305,1.783571,0.369104,0.455175
5165,2020-07-15,111.78,113.65,111.11,112.78,2232700,0,0,105.116333,0.932850,106.334735,61.994711,2.778635,-13.040629,1.804164,0.465205,0.461862
